## Import Libraries

In [2]:
import numpy as np
import cv2
import matplotlib.image as mplimg
import matplotlib.pyplot as plt
import glob
import os
import pickle

%matplotlib qt 

## Camera Calibration
We will compute the calibration matrix and distortion coefficients given a set of chess board images
[tutorial] (https://docs.opencv.org/2.4/doc/tutorials/calib3d/camera_calibration/camera_calibration.html)
Camera Matrix (C) --> Converts 3D real world points P(x,y,z) to 2D image points p(x,y) [pinhole camera model]
P ~ Cp





In [3]:
os.chdir('D:\\Courses\\Udacity\\Workbooks\\ND013\\SelfDrivingCarEngineer\\CarND-Advanced-Lane-Lines')
print(os.listdir())
calimg1 = mplimg.imread(".\camera_cal\calibration1.jpg")

plt.imshow(calimg1)

['.vscode', 'advanced_lane_detection.ipynb', 'calibration_params.p', 'camera_cal', 'challengevidframes', 'challenge_video.mp4', 'examples', 'example_writeup.pdf', 'harderchallengevidframes', 'harder_challenge_video.mp4', 'ldpackage', 'LICENSE', 'output_images', 'pipeline.py', 'processed_challenge_video.mp4', 'processed_harder_challenge_video.mp4', 'processed_project_video.mp4', 'projectvidframes', 'project_video.mp4', 'pTransformData.p', 'README.md', 'run.py', 'set_git.sh', 'test_images', 'writeup_template.md', '__pycache__']


In [3]:
# The number of object points for this project is 9x6
# prepare object points
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

In [4]:
# Curate Image List
images = glob.glob('.\camera_cal\calibration*.jpg')

print(len(images))
# Step through all of the calibration images
for idx,fname in enumerate(images):
    image = cv2.imread(fname) #BGR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #Identify Chessboard Corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # Draw and display the corners
        cv2.drawChessboardCorners(image, (9,6), corners, ret)
        cv2.imshow('Image', image)
        
        cv2.waitKey(500)
cv2.destroyAllWindows()




20


In [6]:
img = cv2.imread(images[0])
img_size = img.shape
print(img_size[0:2])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size[0:2],None,None)
#Save the camera calibration results for later use
calibration = {}
calibration["CameraMatrix"] = mtx
calibration["DistortionCoeff"] = dist
calibration["RotationalVectors"] = rvecs
calibration["TranslationalVectors"] = tvecs
pickle.dump(calibration,open("calibration_params.p","wb"))

# Undistort calibration images
for fname in images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('Undistorted Image', fontsize=30)



(720, 1280)


## Computer Vision Operations

In [4]:
def undistort(image):
    calibration_file = open("calibration_params.p",'rb')
    params = pickle.load(calibration_file)
    CMtx = params["CameraMatrix"]
    distC = params["DistortionCoeff"]
    undistorted_image = cv2.undistort(image, CMtx, distC, None, CMtx)
    return undistorted_image

def gaussian_blur(img, kernel_size=3):
    blurred_image = cv2.GaussianBlur(img,(kernel_size, kernel_size),0)
    return blurred_image

def canny_thresh(img, thresh_low=0, thresh_high=255):
    edges = cv2.Canny(img, thresh_low, thresh_high)
    return edges

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    
    dx=(orient=='x')
    dy=(orient=='y')
    sobel = cv2.Sobel(img,cv2.CV_64F,dx,dy,sobel_kernel)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel<=thresh_max)] = 1
    
    return binary_output

def gradient_thresh(image,sobel_kernel=3,x_thresh=(0,255),y_thresh=(0,255)):
    gradx = abs_sobel_thresh(image, 'x', sobel_kernel, thresh=x_thresh)
    grady = abs_sobel_thresh(image, 'y', sobel_kernel, thresh=y_thresh)
    binary_output = np.zeros_like(image)
    binary_output[(gradx==1) & (grady==1)]=1
    return binary_output



def polar_thresh(image,sobel_kernel,mag_thresh=(0,255),ang_thresh=(0,np.pi/2)):
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,None,sobel_kernel)
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,None,sobel_kernel)
    mag, ang = cv2.cartToPolar(sobelx, sobely)
    scaled_mag = np.uint8(255*mag/np.max(mag))
    binary_output = np.zeros_like(image)
    binary_output[(scaled_mag >= mag_thresh[0]) & (scaled_mag<=mag_thresh[1]) & (ang >= ang_thresh[0]) & (ang<=ang_thresh[1])] = 1
    binary_output_debug_mag = np.zeros_like(image)
    binary_output_debug_mag[(scaled_mag >= mag_thresh[0]) & (scaled_mag<=mag_thresh[1])]=1
    binary_output_debug_ang = np.zeros_like(image)
    binary_output_debug_ang[(ang >= ang_thresh[0]) & (ang<=ang_thresh[1])]=1

    # plt.figure()
    # plt.imshow(scaled_mag, cmap='gray')
    # plt.figure()
    # plt.imshow(binary_output_debug_mag, cmap='gray')
    # plt.figure()
    # plt.imshow(binary_output_debug_ang, cmap='gray')

    return binary_output
def color_thresh(image,ch=1,thresh=(0,255)):
    #Assume a BGR image is being passed in
    channel = image[:,:,ch] # First Channel    
    binary_output = np.zeros_like(channel)
    binary_output[(channel>thresh[0]) & (channel<=thresh[1])]=1
    return binary_output

def S_thresh(image, thresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s = hls[:,:,2]
    binary_output = np.zeros_like(s)
    binary_output[(s>=thresh[0]) & (s<=thresh[1])]=1
    return binary_output


def warp(image):
    transform_file = open("pTransformData.p",'rb')
    params = pickle.load(transform_file)
    M = params["M"]
    warped = cv2.warpPerspective(image,M,image.shape[1::-1])   
    return warped

def unwarp(image):
    transform_file = open("pTransformData.p",'rb')
    params = pickle.load(transform_file)
    Minv = params["Minv"]
    unwarped = cv2.warpPerspective(image,Minv,image.shape[1::-1])    
    return unwarped

## Perspective Transform Computation

In [6]:
cv2.destroyAllWindows()
plt.close('all')
test_images = glob.glob('.\\test_images\\*.jpg')   

subset=[test_images[0]]
for fname in test_images:
    img = cv2.imread(fname)
    
    undistorted_image = undistort(img)
    
    gray = cv2.cvtColor(undistorted_image,cv2.COLOR_BGR2GRAY)
    filtered = gaussian_blur(gray,kernel_size=5)
    edges = canny_thresh(filtered,thresh_low=100,thresh_high=240)
    # gradthresh = gradient_thresh(filtered,sobel_kernel=3,x_thresh = (20,100), y_thresh =(20,100))
    gradthresh = abs_sobel_thresh(filtered,orient='x', sobel_kernel=5,thresh=(50,200))
    polarthresh = polar_thresh(filtered, sobel_kernel=3, mag_thresh=(100, 255), ang_thresh=(np.pi/4,60*np.pi/180))
    colorthresh = color_thresh(undistorted_image,ch=2,thresh=(200,255))
    sthresh = S_thresh(undistorted_image, thresh=(200, 255))
    bin_output = np.zeros_like(gray)
    bin_output[(gradthresh==1) | (sthresh == 1)] = 1
    
    f, ((ax1, ax2),(ax3,ax4),(ax5,ax6),(ax7,ax8)) = plt.subplots(4, 2, figsize=(20,20))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(undistorted_image)
    ax2.set_title('Undistorted Image', fontsize=30)
    ax3.imshow(filtered, cmap = 'gray')
    ax3.set_title('Filtered Image', fontsize=30)
    ax4.imshow(edges, cmap='gray')
    ax4.set_title('Canny Edge Detected Image', fontsize=30)
    ax5.imshow(gradthresh, cmap = 'gray')
    ax5.set_title('Gradient Thresholded Image', fontsize=30)
    ax6.imshow(polarthresh, cmap='gray')
    ax6.set_title('Polar Thresholded Image', fontsize=30)
    plt.tight_layout()
    ax7.imshow(colorthresh, cmap = 'gray')
    ax7.set_title('Color Thresholded Image', fontsize=30)
    ax8.imshow(sthresh, cmap='gray')
    ax8.set_title('Saturation Thresholded Image', fontsize=30)
    plt.tight_layout()
    # plt.savefig(os.path.join('output_images', os.path.basename(fname))+'_visualize_thresh.png')
    
    plt.figure()
    plt.imshow(bin_output,cmap='gray')
    plt.show()

## Warp Image

In [5]:
image = cv2.imread('.\\test_images\straight_lines1.jpg')

undistorted_image = undistort(image)
print(undistorted_image.shape[0:2])
print(undistorted_image.shape[1::-1])
# plt.figure()
# plt.imshow(undistorted_image)
src = np.float32([[475,526],[810,526],[1062,680],[240,680]]) # top left, top right, bottom left, bottom right
plt.figure()
plt.imshow(cv2.polylines(undistorted_image,np.int32(src),True,(0,0,255),3))
plt.suptitle('undistored image')

dst = np.float32([[300,500],[900,500],[900,719],[300,719]]) # Heuristic offset of 100 from each corner
M = cv2.getPerspectiveTransform(src,dst,cv2.INTER_LINEAR)
print(M)
warped = cv2.warpPerspective(undistorted_image,M,undistorted_image.shape[1::-1])
plt.figure()
plt.imshow(cv2.polylines(warped,np.int32(dst),True,(0,0,255),3))
plt.suptitle('undistored image')

Minv = cv2.getPerspectiveTransform(dst,src,cv2.INTER_LINEAR)
unwarped = cv2.warpPerspective(warped,Minv,warped.shape[1::-1])
plt.figure()
plt.imshow(cv2.polylines(unwarped,np.int32(src),True,(0,0,255),3))
plt.suptitle('undistored image')

pTransform = {}
pTransform['M']=M
pTransform['Minv']=Minv
pickle.dump(pTransform,open('pTransformData.p','wb'))



(720, 1280)
(1280, 720)


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2435: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'cv::polylines'


In [9]:
plt.close('all')
test_images = glob.glob('.\\test_images\\*.jpg')   

subset=[test_images[0]]
for fname in test_images:
    image = cv2.imread(fname)
    undistorted_image = undistort(image)
    warped = warp(undistorted_image)
    unwarped = unwarp(warped)
    
    f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(20,20))
    f.suptitle(os.path.basename(fname),fontsize=30)
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(undistorted_image)
    ax2.set_title('Undistorted Image', fontsize=30)
    ax3.imshow(warped)
    ax3.set_title('Warped Image', fontsize=30)
    ax4.imshow(unwarped)
    ax4.set_title('Unwarped Image', fontsize=30)
    plt.tight_layout()
    plt.savefig(os.path.join('output_images', os.path.basename(fname))+'_birds_eye.png')

In [10]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin # Update this
        win_xleft_high = leftx_current + margin   # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
        

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]


    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img


plt.close('all')
test_images = glob.glob('.\\test_images\\*.jpg')   

subset=[test_images[0]]
for fname in test_images:
    image = cv2.imread(fname)
    undistorted_image = undistort(image)
    gray = cv2.cvtColor(undistorted_image,cv2.COLOR_BGR2GRAY)
    filtered = gaussian_blur(gray,kernel_size=5)
    polarthresh = polar_thresh(filtered, sobel_kernel=3, mag_thresh=(100, 255), ang_thresh=(np.pi/4,60*np.pi/180))
    colorthresh = color_thresh(undistorted_image,ch=2,thresh=(200,255))
    sthresh = S_thresh(undistorted_image, thresh=(200, 255))
    bin_output = np.zeros_like(gray)
    bin_output[(colorthresh==1) & ((polarthresh==1) | (sthresh == 1))] = 1
    binary_warped = warp(bin_output)

    out_img = fit_polynomial(binary_warped)

    plt.imshow(out_img)

    plt.show()

LinAlgError: SVD did not converge in Linear Least Squares